In [ ]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent))
# sys.path

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate


In [ ]:
%load_ext autoreload
%autoreload 2
%aimport optiml.queries

## Initialization (Done)

In [ ]:
# Initialize Snowflake connection to Snowflake
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.queries import SNFLKQuery
qlib = SNFLKQuery(connection, 'KIV', cache_dir)
sdate = '2022-10-01'
edate = '2022-10-31'
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

# Total cost breakdown 

## Visualization

## Breakdown of total cost by usage type (Done)

In [ ]:
df = qlib.total_cost_breakdown(sdate, edate).round(2)
df.loc[len(df.index)] = ['Total', df['credits'].sum(), df['dollars'].sum()]
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))


In [ ]:
df = df.drop(len(df)-1) ## Remove the last row of totals for the plot
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{"type": "pie"},{"type": "pie"}]],
    subplot_titles=("Dollars", "Credits")
)

fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['dollars'].tolist(),name="Dollars"),row=1,col=1)
fig.add_trace(go.Pie(labels=df['cost_category'].tolist(), values=df['credits'].tolist(),name='Credits'),row=1,col=2)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by usage category",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'})
fig.show()

## Analysis
- Compute forms overwhelming majority of the cost
- Cloud services are in aggregate < 10 % of compute so are not an added expense
- Autoclustering used to be on in 2020 but isn't anymore
- Should explore if there are any compute efficiencies to be gained by 
    - Autoclustering
    - Materialized views / caching
    - Search optimization
    - Query optimizations / reducing scans

* Largely used by compute
* In 2020 auto clustering was on but was turned off - why?
* Storage credits at 0 since storage is charged as a flat rate - are numbers consistent?
* Definitely did not use $80K worth of credits in October since total bill is $50K - so something is wrong!


## Breakdown of total cost as a time series

In [ ]:
df = qlib.total_cost_breakdown_ts(sdate, edate)

In [ ]:
df.head()

In [ ]:
df[df['category_name'] == 'Storage']

In [ ]:
df_by_user = df.groupby(['user_name','start_time','end_time']).sum('numeric_only').reset_index()
df_by_user.head()

In [ ]:
df_by_category = df.groupby(['category_name','start_time','end_time']).sum('numeric_only').reset_index()
fig = px.area(df_by_category, x="start_time", y="credits", color="category_name",color_discrete_sequence=color_scheme )#, line_group="country")
fig.show()

## Breakdown of total cost by warehouse

In [ ]:
df = qlib.cost_by_wh(sdate, edate).round(2)
df.loc[len(df.index)] = ['Total', df['credits'].sum(), df['dollars'].sum()]
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
df = df.drop(len(df)-1) ## Remove the last row of totals for the plot

fig = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "pie"}]],
)

fig.add_trace(go.Pie(labels=df['warehouse_name'].tolist(), values=df['dollars'].tolist(),name='dollars'),row=1,col=1)

fig.update_layout(
    title={
        'text': "Breakdown of total cost by warehouse",
        'y':0.1,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Analysis
- Sum of the compute across all the warehouses is 1255.47 credits which is consistent with total cost of compute above. Sanity check to ensure we are not missing any compute or warehouse

## Breakdown of cost by partner tools

In [ ]:
df = qlib.cost_by_partner_tool_ts(sdate, edate).round(2)
df = df.drop(len(df)-1).groupby('client_application_name').sum('numeric_only').reset_index()
df.loc[len(df.index)] = ['Total', df['approximate_credits_used'].sum()]
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
df = df.drop(len(df)-1) ## Remove the last row of totals for the plot
fig = make_subplots(
    rows=1, cols=1,
    specs=[[{"type": "pie"}]],
)

fig.add_trace(go.Pie(labels=df_client_app['client_application_name'].tolist(), values=df_client_app['approximate_credits_used'].tolist(),name='credits'),row=1,col=1)

fig.update_layout(
    title={
        'text': "Breakdown of credits used by Client Apps",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## Breakdown of total cost by user (to do)

In [ ]:
df = qlib.cost_by_user_ts(sdate, edate)
df

## Warehouse configuration

In [ ]:
df = qlib.warehouse_config()
print(tabulate(df, headers='keys', tablefmt='rounded_outline', showindex=False))

In [ ]:
df.columns

In [ ]:
df.name

## Analysis
- auto suspend times here vary from 60 min to 3600 min - these seem very large assuming auto resuming does not take very long
- There are no resource monitors being used - this should be corrected

## Cost of queries

In [ ]:
df = qlib.n_expensive_queries(sdate, edate, 10)
df

In [ ]:
df['query_text'][0] == df['query_text'][1]

In [ ]:
df['query_text'][0]

In [ ]:
df['query_text'][1]

In [ ]:
df = qlib.cost_by_wh_ts()
df.warehouse_name.unique()

In [ ]:
df=qlib.idle_users(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.users_full_table_scans(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.heavy_users(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.users_never_logged_in(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.users_never_logged_in(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.idle_roles(start_date="2022-01-01", end_date="2022-02-02")
df.head()


In [ ]:
df=qlib.failed_tasks(start_date="2022-01-01", end_date="2022-02-02")
df.head()

In [ ]:
df=qlib.long_running_tasks(start_date="2022-01-01", end_date="2022-02-02")
df.head()